# Processing results using BigQuery #

We start by importing all the requisite packages from BayesCMD etc. as well as ones required to plot and read data from big query.

In [1]:
import os
import argparse
from pathlib import Path
import json
import matplotlib.pyplot as plt
from distutils import dir_util
from pprint import pprint
import pickle

# BayesCMD packages 
from bayescmd.results_handling import kde_plot
from bayescmd.results_handling import scatter_dist_plot
from bayescmd.results_handling import data_import
from bayescmd.results_handling import plot_repeated_outputs
from bayescmd.results_handling import histogram_plot
from bayescmd.results_handling import data_merge_by_batch
from bayescmd.abc import import_actual_data
from bayescmd.abc import priors_creator

# Google BigQuery
from google.cloud import bigquery
%load_ext google.cloud.bigquery

STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD


In [2]:
# Explicitly use service account credentials by specifying the private
# key file. All clients in google-cloud-python have this helper.
client = bigquery.Client.from_service_account_json(
    "../../gcloud/hypothermia-auth.json"
)

In [3]:
def generate_histogram_query(project, dataset, model, n_bins, distance):
    histogram_query = """
    SELECT
      MIN(data.{distance}) AS min,
      MAX(data.{distance}) AS max,
      COUNT(data.{distance}) AS num,
      INTEGER((data.{distance}-value.min)/(value.max-value.min)*{n_bins}) AS group_
    FROM
      [{project}:{dataset}.{model}] data
    CROSS JOIN (
      SELECT
        MAX({distance}) AS max,
        MIN({distance}) AS min
      FROM
        [{project}:{dataset}.{model}]) value
    GROUP BY
      group_
    ORDER BY
      group_
    """.format(dataset=dataset, model=model, n_bins=n_bins, distance=distance, project=project)
    return histogram_query

In [4]:
def generate_posterior_query(project, dataset, model, distance, parameters, limit=50000):
    unpacked_params = ",\n".join(parameters)
    histogram_query = """
SELECT
    {unpacked_params},
    {distance},
    idx
FROM
  `{project}.{dataset}.{model}`
ORDER BY
  {distance} ASC
LIMIT
  {limit}
    """.format(project=project, dataset=dataset, model=model, unpacked_params=unpacked_params,distance=distance, limit=limit)
    return histogram_query

In [5]:
def load_configuration(model_version, dataset, verbose=False):
    current_file = Path(os.path.abspath(''))
    config_file = os.path.join(current_file.parents[2],
                              'config_files',
                               'abc',
                               'bp_hypothermia_{}'.format(model_version),
                               'bp_hypothermia_{}_config.json'.format(model_version)
                              )

    with open(config_file, 'r') as conf_f:
        conf = json.load(conf_f)

    params = conf['priors']

    input_path = os.path.join(current_file.parents[2],
                              'data',
                              'clean_hypothermia',
                              '{}_filtered_formatted.csv'.format(dataset.upper()))

    d0 = import_actual_data(input_path)

    targets = conf['targets']
    model_name = conf['model_name']
    inputs = conf['inputs']

    config = {
        "model_name": model_name,
        "targets": targets,
        "inputs": inputs,
        "parameters": params,
        "input_path": input_path,
        "zero_flag": conf['zero_flag'],
    }
    
    if verbose:
        pprint(config)
        
    return config, d0

In [6]:
configuration = {}
# combinations = [('1', 'LWP475'), ('1_1', 'LWP479'),
#                 ('2', 'LWP475'), ('2_1', 'LWP479'),
#                 ('4', 'LWP475'), ('4_1', 'LWP479')]

combinations = [('0', 'LWP475'), ('0', 'LWP479')]


for combo in combinations:
    
    model_number = combo[0]
    DATASET = combo[1]

    model_name = 'bph{}'.format(model_number)
    if model_name not in configuration.keys():
        configuration[model_name] = {}

    configuration[model_name][DATASET] = {}
    print(configuration[model_name].keys())
    config, d0 = load_configuration(model_number, DATASET)
    configuration[model_name][DATASET]['bayescmd_config'] = config
    configuration[model_name][DATASET]['original_data']= d0

    configuration[model_name][DATASET]['histogram_query'] = generate_histogram_query('hypothermia-bayescmd', 
                                                                                        DATASET, 
                                                                                        model_name, 
                                                                                        100, 
                                                                                        'NRMSE')

    configuration[model_name][DATASET]['posterior_query'] = generate_posterior_query('hypothermia-bayescmd', 
                                                                                        DATASET, 
                                                                                        model_name, 
                                                                                        'NRMSE', 
                                                                                        list(configuration[model_name][DATASET]['bayescmd_config']['parameters'].keys()))


dict_keys(['LWP475'])
dict_keys(['LWP475', 'LWP479'])


In [7]:
configuration['bph0'].keys()

dict_keys(['LWP475', 'LWP479'])

In [8]:
import string
labels = {"t": "Time (sec)",
              "SaO2sup": "SaO2 (%)",
              "P_a": "ABP (mmHg)",
              "temp": "Temperature ($^{\circ}$C)",
               "TOI": "TOI (%)",
              "HbO2": "$\Delta$HbO$_2$ $(\mu M)$",
              "HHb": "$\Delta$HHb $(\mu M)$",
              "CCO": "$\Delta$CCO $(\mu M)$"
             }
LIM=50000
# combinations = [('bph1', 'LWP475'), ('bph1_1', 'LWP479'),
#                 ('bph2', 'LWP475'), ('bph2_1', 'LWP479'),
#                 ('bph4', 'LWP475'), ('bph4_1', 'LWP479')]

combinations = [('bph0', 'LWP475'), ('BPH0', 'LWP479')]

signals = ['CCO', 'HbO2', 'HHb']
# for model_name in ['bph2_1']:
#     for DATASET in ['LWP479']:#, 'LWP479' ,'LWP481' ,'LWP484']:
for fig_num, combo in enumerate(combinations):
    DATASET=combo[1]
    model_name=combo[0]
    print("Working on {} - {}".format(model_name, DATASET))
    # Set config and create figure path
    config = configuration[model_name][DATASET]['bayescmd_config']
    figPath = "/home/buck06191/Dropbox/phd/hypothermia/ABC/Figures/{}/{}/{}".format(model_name, DATASET, 'NRMSE')
    dir_util.mkpath(figPath)

    # Get posterior
    print("\tRunning SQL query")
    df_post = client.query(configuration[model_name][DATASET]['posterior_query']).to_dataframe()

    # Plot posterior 
    print("\tPlotting posterior")
    g = kde_plot(df_post, config['parameters'], limit=LIM, n_ticks=4, d="NRMSE", median_file=os.path.join(figPath, "medians.txt"))
    g.fig._suptitle.set_text(g.fig._suptitle.get_text() + 'for {} - {}'.format(DATASET, model_name))
    with open(os.path.join(figPath, 'posterior_{}_{}.pickle'.format(model_name, DATASET)), 'wb') as f: 
        pickle.dump(g, f)
    g.fig.savefig(
        os.path.join(figPath, 'posterior_{}_{}.png'
                     .format(model_name, DATASET)),
        bbox_inches='tight', dpi=250)

    # Plot posterior predictive
    config["offset"] = {}
    print("\tPlotting Posterior Predictive")
    fig, axes, lgd = plot_repeated_outputs(df_post, n_repeats=5000, limit=LIM,
                                    distance='NRMSE', **config)
    for ix, ax in enumerate(axes.flatten()):
        ax.set_ylabel(labels[signals[ix]])
        old_title = ax.get_title().split(':')
        new_title = ":".join([labels[signals[ix]].split()[0], old_title[1]])
        ax.set_title(new_title, size=11)
    axes.flatten()[-1].set_xlabel(labels['t'])
    fig.suptitle(string.ascii_lowercase[fig_num]+")", ha='left', x=-0.02, y=0.925)
    #fig.set_size_inches(18.5, 12.5)
    with open(os.path.join(figPath, 'posterior_predictive_{}_{}.pickle'.format(model_name, DATASET)), 'wb') as f: 
        pickle.dump(fig, f)
    fig.savefig(
        os.path.join(figPath, 'posterior_predictive_{}_{}.png'
                     .format(model_name, DATASET)),
        bbox_inches='tight', bbox_extra_artists=(lgd,), dpi=250)
    plt.close('all')

Working on bph0 - LWP475
	Running SQL query
	Plotting posterior
	Plotting Posterior Predictive
Sample 0, idx:10226
Sample 1, idx:39571
Timed out for Sample 1, idx:39571
{'K_sigma': 9.202109698045962,
 'NADpool': 3.2401518981924773,
 'P_ic': 5.356156165681496,
 'R_autc': 1.2983557900114728,
 'Vol_mit': 0.060231929830756266,
 'k_aut': 0.2775064888662161,
 'k_lk2': 0.03564036374785187,
 'n_m': 1.781863852034807,
 'phi': 0.03663129788178657,
 'r_0': 0.010092089249682051,
 'r_m': 0.026002504731920548,
 'r_t': 0.015285420434515478,
 'rocc': 0.8866882234078329}
Sample 1, idx:38748
Timed out for Sample 1, idx:38748
{'K_sigma': 10.773796911311939,
 'NADpool': 2.941291463329206,
 'P_ic': 4.437331145891815,
 'R_autc': 8.253973670729739,
 'Vol_mit': 0.06808962943851724,
 'k_aut': 0.673171657764054,
 'k_lk2': 0.045474921193671435,
 'n_m': 1.6547777874140694,
 'phi': 0.04251540369134976,
 'r_0': 0.010169472897119674,
 'r_m': 0.025549692439066168,
 'r_t': 0.015806575939310362,
 'rocc': 0.802882493043

Sample 89, idx:25310
Sample 90, idx:26216
Sample 91, idx:29299
Sample 92, idx:34404
Sample 93, idx:19004
Sample 94, idx:5669
Sample 95, idx:43969
Timed out for Sample 95, idx:43969
{'K_sigma': 11.702200479031793,
 'NADpool': 3.177283469043733,
 'P_ic': 4.757122618449436,
 'R_autc': 5.671741874903019,
 'Vol_mit': 0.05603325694444628,
 'k_aut': 0.22365350629912306,
 'k_lk2': 0.04264339354476043,
 'n_m': 1.6997895961168978,
 'phi': 0.03641313458394628,
 'r_0': 0.010160919245547632,
 'r_m': 0.02433933212386168,
 'r_t': 0.01566166809755942,
 'rocc': 0.7274103682349004}
Sample 95, idx:29351
Sample 96, idx:19369
Sample 97, idx:8097
Sample 98, idx:26077
Sample 99, idx:10312
Sample 100, idx:7656
Sample 101, idx:8127
Sample 102, idx:44909
Sample 103, idx:19607
Sample 104, idx:2437
Sample 105, idx:44
Sample 106, idx:2139
Sample 107, idx:38395
Sample 108, idx:29992
Sample 109, idx:17824
Sample 110, idx:42222
Sample 111, idx:29082
Sample 112, idx:6334
Sample 113, idx:32874
Sample 114, idx:49613
Sam

Timed out for Sample 225, idx:35057
{'K_sigma': 11.160103674779434,
 'NADpool': 3.245386207995232,
 'P_ic': 4.764340823237947,
 'R_autc': 7.8501695574356924,
 'Vol_mit': 0.06445088727607927,
 'k_aut': 0.2031111523970496,
 'k_lk2': 0.0353818808928336,
 'n_m': 1.524253286337767,
 'phi': 0.04165745265707063,
 'r_0': 0.01080120607513605,
 'r_m': 0.02369496037315773,
 'r_t': 0.01539970758561746,
 'rocc': 0.7602333423510893}
Sample 225, idx:16260
Sample 226, idx:26558
Sample 227, idx:16559
Sample 228, idx:42128
Sample 229, idx:37998
Sample 230, idx:18173
Sample 231, idx:38742
Sample 232, idx:24741
Timed out for Sample 232, idx:24741
{'K_sigma': 9.128402615738032,
 'NADpool': 2.716888634703116,
 'P_ic': 3.793936819100635,
 'R_autc': 9.997797806921511,
 'Vol_mit': 0.07203792919709892,
 'k_aut': 0.4691153812628828,
 'k_lk2': 0.04012778226777783,
 'n_m': 1.9500980221963984,
 'phi': 0.04154222122391435,
 'r_0': 0.010482652519673341,
 'r_m': 0.024595272601899087,
 'r_t': 0.014911368298281196,
 'ro

Timed out for Sample 302, idx:26796
{'K_sigma': 8.053688296824681,
 'NADpool': 3.556870210135937,
 'P_ic': 3.7573094537202585,
 'R_autc': 2.3496119396690074,
 'Vol_mit': 0.0597764583689661,
 'k_aut': 0.41622860114918736,
 'k_lk2': 0.042433998702758775,
 'n_m': 1.4769026135246093,
 'phi': 0.04305616203242146,
 'r_0': 0.010265008571110484,
 'r_m': 0.02306073776194258,
 'r_t': 0.014792183770873919,
 'rocc': 0.13307674960106552}
Sample 302, idx:6669
Sample 303, idx:8317
Sample 304, idx:32058
Sample 305, idx:49556
Sample 306, idx:6673
Sample 307, idx:22652
Timed out for Sample 307, idx:22652
{'K_sigma': 9.543955474532119,
 'NADpool': 2.7719273486188065,
 'P_ic': 4.617868176572647,
 'R_autc': 2.177929304656216,
 'Vol_mit': 0.06832251313518327,
 'k_aut': 0.5025980411260532,
 'k_lk2': 0.03407711324820625,
 'n_m': 2.0687586583780044,
 'phi': 0.043130774817767135,
 'r_0': 0.010977559153536771,
 'r_m': 0.024938467528062856,
 'r_t': 0.014597306103981612,
 'rocc': 0.4940530479715529}
Sample 307, id

Sample 412, idx:40751
Sample 413, idx:38267
Sample 414, idx:9772
Sample 415, idx:2923
Sample 416, idx:38333
Sample 417, idx:28444
Sample 418, idx:7391
Sample 419, idx:28200
Sample 420, idx:24207
Timed out for Sample 420, idx:24207
{'K_sigma': 9.112254043009091,
 'NADpool': 3.790683076345009,
 'P_ic': 4.382082953508526,
 'R_autc': 8.085333187756742,
 'Vol_mit': 0.06526007796818539,
 'k_aut': 0.3980755041601936,
 'k_lk2': 0.03638606687339038,
 'n_m': 1.9337517264602104,
 'phi': 0.04011241536546197,
 'r_0': 0.011651004068506688,
 'r_m': 0.024481405013274917,
 'r_t': 0.014646181249415397,
 'rocc': 0.2980453415371327}
Sample 420, idx:17256
Sample 421, idx:26186
Sample 422, idx:13567
Sample 423, idx:6281
Sample 424, idx:14923
Sample 425, idx:18845
Timed out for Sample 425, idx:18845
{'K_sigma': 11.185263966159546,
 'NADpool': 3.3448806306063266,
 'P_ic': 5.310569820191225,
 'R_autc': 2.7818859680458496,
 'Vol_mit': 0.07110679275159744,
 'k_aut': 0.36844636612489945,
 'k_lk2': 0.0325735132117

...

Sample 974, idx:32586
Sample 975, idx:45929
Sample 976, idx:15354
Sample 977, idx:42129
Sample 978, idx:49497
Sample 979, idx:45173
Sample 980, idx:3915
Sample 981, idx:40118
Sample 982, idx:11226
Sample 983, idx:48019
Sample 984, idx:39002
Sample 985, idx:10263
Sample 986, idx:12763
Sample 987, idx:15441
Timed out for Sample 987, idx:15441
{'K_sigma': 10.707997592235227,
 'NADpool': 2.752365763936173,
 'P_ic': 4.220860332685566,
 'R_autc': 1.5704085538109436,
 'Vol_mit': 0.07974654223866119,
 'k_aut': 0.4692415182754103,
 'k_lk2': 0.03357087158471067,
 'n_m': 1.7660980990008233,
 'phi': 0.04220675239409608,
 'r_0': 0.011039644287616564,
 'r_m': 0.028542620854306863,
 'r_t': 0.015001941287954488,
 'rocc': 0.7816371223883901}
Sample 987, idx:16848
Sample 988, idx:17308
Sample 989, idx:21812
Sample 990, idx:39211
Sample 991, idx:7628
Sample 992, idx:34825
Sample 993, idx:29405
Sample 994, idx:43886
Timed out for Sample 994, idx:43886
{'K_sigma': 11.704265104535134,
 'NADpool': 3.51230243

Sample 1120, idx:45624
Sample 1121, idx:7892
Timed out for Sample 1121, idx:7892
{'K_sigma': 10.749112769764901,
 'NADpool': 2.702497626189033,
 'P_ic': 3.9863783441267215,
 'R_autc': 7.0143236446326425,
 'Vol_mit': 0.06489770683537142,
 'k_aut': 0.5935159568103652,
 'k_lk2': 0.042494024486813636,
 'n_m': 1.9984583028234209,
 'phi': 0.0406020225928083,
 'r_0': 0.010424625056335606,
 'r_m': 0.03153803377055223,
 'r_t': 0.014582543647154418,
 'rocc': 0.7895357101954557}
Sample 1121, idx:24138
Sample 1122, idx:46919
Sample 1123, idx:7611
Sample 1124, idx:48423
Sample 1125, idx:7405
Sample 1126, idx:18409
Sample 1127, idx:2135
Sample 1128, idx:15501
Sample 1129, idx:41905
Sample 1130, idx:22177
Sample 1131, idx:9417
Sample 1132, idx:32064
Sample 1133, idx:47170
Sample 1134, idx:2152
Sample 1135, idx:25479
Timed out for Sample 1135, idx:25479
{'K_sigma': 10.723979542151797,
 'NADpool': 3.8656942825475116,
 'P_ic': 5.105150410349473,
 'R_autc': 1.0237962095333586,
 'Vol_mit': 0.0655329732804

Sample 1199, idx:37577
Sample 1200, idx:28227
Sample 1201, idx:11219
Sample 1202, idx:18527
Timed out for Sample 1202, idx:18527
{'K_sigma': 10.438305216577133,
 'NADpool': 3.0964851798471638,
 'P_ic': 4.6950345880208495,
 'R_autc': 0.8200291048727826,
 'Vol_mit': 0.06338188812669387,
 'k_aut': 0.4171388576691647,
 'k_lk2': 0.03696998186854731,
 'n_m': 1.8119183460836392,
 'phi': 0.04163441373644741,
 'r_0': 0.010159826870088387,
 'r_m': 0.03166826025792331,
 'r_t': 0.015229002959492827,
 'rocc': 0.3005798551677832}
Sample 1202, idx:48484
Sample 1203, idx:11754
Sample 1204, idx:20500
Sample 1205, idx:37594
Sample 1206, idx:6702
Sample 1207, idx:39722
Sample 1208, idx:19522
Sample 1209, idx:33663
Sample 1210, idx:24055
Timed out for Sample 1210, idx:24055
{'K_sigma': 10.763194749345029,
 'NADpool': 3.7522077239806753,
 'P_ic': 4.426793423302058,
 'R_autc': 2.2576776440618307,
 'Vol_mit': 0.0544879876046654,
 'k_aut': 0.5996463381645254,
 'k_lk2': 0.042957221045160664,
 'n_m': 2.08941910

Sample 1307, idx:13393
Sample 1308, idx:5049
Sample 1309, idx:22698
Sample 1310, idx:15254
Sample 1311, idx:4492
Sample 1312, idx:8270
Timed out for Sample 1312, idx:8270
{'K_sigma': 11.386526346297352,
 'NADpool': 3.0797916464362487,
 'P_ic': 5.319347074077486,
 'R_autc': 1.4965385081219518,
 'Vol_mit': 0.0678990642601089,
 'k_aut': 0.4753755547830924,
 'k_lk2': 0.041186589206455834,
 'n_m': 1.717921972986471,
 'phi': 0.040172025024495586,
 'r_0': 0.011672162760185014,
 'r_m': 0.02464044614751193,
 'r_t': 0.01478942810140209,
 'rocc': 0.252461155325988}
Sample 1312, idx:35378
Sample 1313, idx:2161
Sample 1314, idx:2369
Sample 1315, idx:2187
Sample 1316, idx:20628
Sample 1317, idx:5802
Sample 1318, idx:10574
Timed out for Sample 1318, idx:10574
{'K_sigma': 9.550951284690068,
 'NADpool': 2.7464756289432737,
 'P_ic': 5.155769247409858,
 'R_autc': 5.307480035979003,
 'Vol_mit': 0.07342235061240603,
 'k_aut': 0.4988244012984498,
 'k_lk2': 0.03947976420434378,
 'n_m': 1.5999110565360473,
 '

KeyboardInterrupt: 

In [ ]:
# fig.subplots_adjust(bottom=0.2)
# for ax in fig.axes:
#     ax.set_ylabel("$\Delta$" + ax.get_ylabel()+" ($\mu$M)")
# with open(os.path.join(figPath, 'posterior_predictive_{}_{}.pickle'.format(model_name, DATASET)), 'wb') as f: 
#             pickle.dump(fig, f)
# fig.savefig(
#     os.path.join(figPath, 'posterior_predictive_{}_{}.png'
#                  .format(model_name, DATASET)),
#     bbox_inches='tight', bbox_extra_artists=(lgd,), dpi=250)

In [ ]:
# configuration = {}
# for model_number in [1, 2, 4]:
#     model_name = 'bph{}'.format(model_number)
#     configuration[model_name] = {}
#     for DATASET in ['LWP475', 'LWP479' ,'LWP481' ,'LWP484']: 
#         configuration[model_name][DATASET] = {}
#         for output in ['CCO', 'HbO2', 'HHb']:
#             configuration[model_name][DATASET][output] = {}
#             config, d0 = load_configuration(model_number, DATASET)
#             configuration[model_name][DATASET][output]['bayescmd_config'] = config
#             configuration[model_name][DATASET][output]['original_data']= d0

#             configuration[model_name][DATASET][output]['histogram_query'] = generate_histogram_query('hypothermia-bayescmd', 
#                                                                                                 DATASET, 
#                                                                                                 model_name, 
#                                                                                                 100, 
#                                                                                                 '{}_NRMSE'.format(output))

#             configuration[model_name][DATASET][output]['posterior_query'] = generate_posterior_query('hypothermia-bayescmd', 
#                                                                                                 DATASET, 
#                                                                                                 model_name, 
#                                                                                                 '{}_NRMSE'.format(output), 
#                                                                                                 list(configuration[model_name][DATASET][output]['bayescmd_config']['parameters'].keys()))


In [ ]:
# %%capture individual_outputs
# LIM=5000
# for model_name in ['bph1', 'bph2', 'bph4']:
#     for DATASET in ['LWP475', 'LWP479' ,'LWP481' ,'LWP484']:
#         for output in ['CCO', 'HbO2', 'HHb']:
#             print("Working on {} - {}, {}\n".format(model_name, DATASET, output))
#             # Set config and create figure path
#             config = configuration[model_name][DATASET][output]['bayescmd_config']
#             figPath = "/home/buck06191/Dropbox/phd/hypothermia/ABC/Figures/{}/{}/{}".format(model_name, DATASET, '{}_NRMSE'.format(output))
#             dir_util.mkpath(figPath)

#             # Get posterior
#             df_post = client.query(configuration[model_name][DATASET][output]['posterior_query']).to_dataframe()

#             # Plot posterior 
#             g = kde_plot(df_post, config['parameters'], limit=LIM, n_ticks=4, d="{}_NRMSE".format(output), median_file=os.path.join(figPath, "medians.txt"))
#             g.fig._suptitle.set_text(g.fig._suptitle.get_text() + 'for {} - {}, {}'.format(DATASET, model_name,output))
#             g.fig._suptitle.set_fontsize(26)
#             with open(os.path.join(figPath, 'posterior_{}_{}_{}.pickle'.format(model_name, DATASET,output)), 'wb') as f: 
#                 pickle.dump(g, f)
#             g.fig.savefig(
#                 os.path.join(figPath, 'posterior_{}_{}_{}.png'
#                              .format(model_name, DATASET,output)),
#                 bbox_inches='tight', dpi=250)

#             # Plot posterior predictive
#             config["offset"] = {}
#             fig, ax, lgd = plot_repeated_outputs(df_post, n_repeats=100, limit=LIM,
#                                             distance='{}_NRMSE'.format(output), **config)
#             fig.suptitle('Posterior predictive distributions\nfor {} - {}, {}'.format(DATASET, model_name,output), fontsize=26)
#             fig.set_size_inches(18.5, 12.5)
#             with open(os.path.join(figPath, 'posterior_predictive_{}_{}_{}.pickle'.format(model_name, DATASET, output)), 'wb') as f: 
#                 pickle.dump(fig, f)
#             fig.savefig(
#                 os.path.join(figPath, 'posterior_predictive_{}_{}_{}.png'
#                              .format(model_name, DATASET,output)),
#                 bbox_inches='tight', bbox_extra_artists=(lgd,), dpi=250)
#             plt.close('all')

In [ ]:
string.ascii_lowercase[0]